In [ ]:
# Pip install big query for pulling publically available data via python
pip install google-cloud-bigquery

In [ ]:
# Install a needed dependency for the .to_dataframe() function 
pip install pyarrow

In [2]:
# only import bigquery from google cloud
from google.cloud import bigquery

In [3]:
# Changing the settings to override this file path in the environment
import os

# Needing to have the key file downloaded to access via the api. The query can be used
# instead and utilized for this brief analysis.
os.environ["GOOGLE_APPLICATION_CREDENTIALS"]="/Users/tiareina/Downloads/compact-scene-317315-b4b3cd5c140c.json"

In [20]:
# Construct a BigQuery client object.
client = bigquery.Client()

query = """
    SELECT usertype, count(*) as RideCount
    FROM `bigquery-public-data.new_york_citibike.citibike_trips`
    GROUP BY usertype
"""
query_job = client.query(query)  # Make an API request.

In [26]:
# collect results as a dataframe
df = query_job.to_dataframe()
df.usertype = df.usertype.apply(lambda x: x.replace('','None') if x == '' else x)
df.RideCount = df.RideCount.astype(int)
df.sort_values(by='RideCount').head()

,usertype,RideCount
0,None,5828994
1,Customer,6191149
2,Subscriber,46917572


In [28]:
# Construct a BigQuery client object.
client = bigquery.Client()

query2 = """
    SELECT usertype,AVG(tripduration) as Average_Ride_Time
    FROM `bigquery-public-data.new_york_citibike.citibike_trips`
    GROUP BY usertype
"""
query_job2 = client.query(query2)  # Make an API request.

,usertype,Average_Ride_Time
0,Subscriber,806.381888
1,Customer,2145.512075
2,,NaN


In [30]:
# collect results as a dataframe
df2 = query_job2.to_dataframe()
df2 = df2[df2['usertype']!='']

#Calculates the average ride time in minutes instead of seconds
df2['Avg_RT_Min'] = df2['Average_Ride_Time'] / 60

In [31]:
df2

,usertype,Average_Ride_Time,Avg_RT_Min
0,Subscriber,806.381888,13.439698
1,Customer,2145.512075,35.758535


In [32]:
# Construct a BigQuery client object.
client = bigquery.Client()

query3 = """
    SELECT usertype,
    start_station_id,
    end_station_id,
    count(*) AS Count_of_Trips
    FROM `bigquery-public-data.new_york_citibike.citibike_trips`
    WHERE stoptime > DATE('2017-01-01')
    GROUP BY usertype, start_station_id, end_station_id
    ORDER BY Count_of_Trips desc;
"""
query_job3 = client.query(query3)  # Make an API request.

In [33]:
# collect results as a dataframe
df3 = query_job3.to_dataframe()

In [34]:
df3

,usertype,start_station_id,end_station_id,Count_of_Trips
0,Subscriber,432,3263,9614
1,Customer,2006,3282,6783
2,Customer,2006,2006,6662
3,Customer,281,281,5805
4,Subscriber,435,509,5666
...,...,...,...,...
504631,Subscriber,426,3052,1
504632,Subscriber,3538,402,1
504633,Subscriber,486,353,1
504634,Customer,3490,3170,1
